In [3]:
from Bio import SeqIO, motifs
from Bio.Seq import Seq
import pandas as pd
import numpy as np
import csv
from Bio.SeqRecord import SeqRecord
import torch
import json
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import StandardScaler
import itertools
from collections import Counter

In [35]:
training = pd.read_csv('training.csv')
training['Protein'] = ''
training

,Sequence,Label,Protein
0,MENEKENLFCEPHKRGLMKTPLKESTTANIVLAEIQPDFGPLTTPT...,1,
1,MDSSCHNATTKMLATAPARGNMMSTSKPLAFSIERIMARTPEPKAL...,1,
2,MKRRQKRKHLENEESQETAEKGGGMSKSQEDALQPGSTRVAKGWSQ...,1,
3,MARVGPGRAGVSCQGRGRGRGGSGQRRPPTWEISDSDAEDSAGSEA...,1,
4,MAAADGGGPGGASVGTEEDGGGVGHRTVYLFDRREKESELGDRPLQ...,1,
...,...,...,...
2513,MEQGKGLAVLILAIILLQGTLAQSIKGNHLVKVYDYQEDGSVLLTC...,0,
2514,MARGPGLAPPPLRLPLLLLVLAAVTGHTAAQDNCTCPTNKMTVCSP...,0,
2515,MELSWHVVFIALLSFSCWGSDWESDRNFISTAGPLTNDLLHNLSGL...,0,
2516,MWCIVLFSLLAWVYAEPTMYGEILSPNYPQAYPSEVEKSWDIEVPE...,0,


In [36]:
fasta_matrix_pos = f"training/human_train_positive.fasta"
fasta_matrix_neg = f"training/human_train_negative.fasta"
with open(fasta_matrix_pos, "r") as file1:
    sequences_pos = list(SeqIO.parse(file1, "fasta"))
    
with open(fasta_matrix_neg, "r") as file2:
    sequences_neg = list(SeqIO.parse(file2, "fasta"))
    
sequences = sequences_pos + sequences_neg

In [ ]:
for i in range(len(sequences)):
    line = training[training['Sequence'] == sequences[i].seq].index
    training['Protein'][line]=sequences[i].name

In [52]:
#training.to_csv("DeepFRI/training_deepfri.csv")

In [54]:
testing = pd.read_csv('testing.csv')
testing['Protein'] = ''
testing

,Sequence,Label,Protein
0,MAELKSLSGDAYLALSHGYAAAAAGLAYGAAREPEAARGYGTPGPG...,1,
1,MPDPAKSAPAPKKGSKKAVTKAQKKDGKKRKRSRKESYSIYVYKVL...,1,
2,MYQSLALAASPRQAAYADSGSFLHAPGAGSPMFVPPARVPSMLSYL...,1,
3,MNMEGLVMFQDLSIDFSQEEWECLDAAQKDLYRDVMMENYSSLVSL...,1,
4,MTRSCSAVGCSTRDTVLSRERGLSFHQFPTDTIQRSKWIRAVNRVD...,1,
...,...,...,...
625,MPLAQLADPWQKMAVESPSDSAENGQQIMDEPMGEEEINPQTEEVS...,0,
626,MAILMLSLQLILLLIPSISHEAHKTSLSSWKHDQDWANVSNMTFSN...,0,
627,MAQASPPRPERVLGASSPEARPAQEALLLPTGVFQVAEKMEKRTCA...,0,
628,MGKSEGPVGMVESAGRAGQKRPGFLEGGLLLLLLLVTAALVALGVL...,0,


In [55]:
fasta_matrix_pos = f"testing/human_test_positive.fasta"
fasta_matrix_neg = f"testing/human_test_negative.fasta"
with open(fasta_matrix_pos, "r") as file1:
    sequences_pos = list(SeqIO.parse(file1, "fasta"))
    
with open(fasta_matrix_neg, "r") as file2:
    sequences_neg = list(SeqIO.parse(file2, "fasta"))
    
sequences = sequences_pos + sequences_neg

In [ ]:
for i in range(len(sequences)):
    line = testing[testing['Sequence'] == sequences[i].seq].index
    testing['Protein'][line]=sequences[i].name

In [58]:
testing

,Sequence,Label,Protein
0,MAELKSLSGDAYLALSHGYAAAAAGLAYGAAREPEAARGYGTPGPG...,1,sp|Q8NDY6|BHE23_HUMAN
1,MPDPAKSAPAPKKGSKKAVTKAQKKDGKKRKRSRKESYSIYVYKVL...,1,sp|P23527|H2B1O_HUMAN
2,MYQSLALAASPRQAAYADSGSFLHAPGAGSPMFVPPARVPSMLSYL...,1,sp|Q9BWX5|GATA5_HUMAN
3,MNMEGLVMFQDLSIDFSQEEWECLDAAQKDLYRDVMMENYSSLVSL...,1,sp|A8MQ14|ZN850_HUMAN
4,MTRSCSAVGCSTRDTVLSRERGLSFHQFPTDTIQRSKWIRAVNRVD...,1,sp|Q9H5L6|THAP9_HUMAN
...,...,...,...
625,MPLAQLADPWQKMAVESPSDSAENGQQIMDEPMGEEEINPQTEEVS...,0,sp|P51812|KS6A3_HUMAN
626,MAILMLSLQLILLLIPSISHEAHKTSLSSWKHDQDWANVSNMTFSN...,0,sp|Q6UY13|YB003_HUMAN
627,MAQASPPRPERVLGASSPEARPAQEALLLPTGVFQVAEKMEKRTCA...,0,sp|Q9UIL8|PHF11_HUMAN
628,MGKSEGPVGMVESAGRAGQKRPGFLEGGLLLLLLLVTAALVALGVL...,0,sp|Q495T6|MMEL1_HUMAN


In [59]:
#testing.to_csv("DeepFRI/testing_deepfri.csv")

In [4]:
def amino_acid_composition(protein_sequence):

    amino_acids = "ACDEFGHIKLMNPQRSTVWY"
    
    count_list = {aa: 0 for aa in amino_acids}
    
    for aa in protein_sequence:
        if aa in count_list:
            count_list[aa] += 1
    total_aa = len(protein_sequence)
    aa_composition = {aa: (count / total_aa) * 100 for aa, count in count_list.items()}

    return aa_composition

In [24]:
def calculate_asdc(protein_sequence, max_skip):

    amino_acids = "ACDEFGHIKLMNPQRSTVWY"

    skip_dipeptides = []
    for i in range(max_skip + 1):
        skip_dipeptides.extend([''.join(pair) for pair in itertools.product(amino_acids, repeat=2)])

    skip_dipeptide_counts = Counter({sd: 0 for sd in skip_dipeptides})
    
    for i in range(len(protein_sequence) - 1):
        for skip in range(max_skip + 1):
            if i + skip + 1 < len(protein_sequence):
                dipeptide = protein_sequence[i] + protein_sequence[i + skip + 1]
                skip_dipeptide_counts[dipeptide] += 1
    
    total_count = sum(skip_dipeptide_counts.values())
    asdc = {sd: count / total_count for sd, count in skip_dipeptide_counts.items()}

    return asdc

In [1]:
def calculate_paac(protein_sequence, lambda_val=2, weight=0.05):

    amino_acids = "ACDEFGHIKLMNPQRSTVWY"
    
    paac = {f'PAAC_{aa}': 0 for aa in amino_acids}
    
    for aa in protein_sequence:
        paac[f'PAAC_{aa}'] += 1
    total_count = len(protein_sequence)
    for aa in amino_acids:
        paac[f'PAAC_{aa}'] /= total_count
    
    sequence_order_effect = [0] * (lambda_val)
    for i in range(1, lambda_val + 1):
        for j in range(len(protein_sequence) - i):
            aa1 = protein_sequence[j]
            aa2 = protein_sequence[j + i]
            sequence_order_effect[i-1] += (ord(aa1) - ord(aa2)) ** 2
        sequence_order_effect[i-1] = sequence_order_effect[i-1] / (len(protein_sequence) - i)
    
    sequence_order_effect = [weight * soe for soe in sequence_order_effect]
    
    for i, soe in enumerate(sequence_order_effect, start=1):
        paac[f'PAAC_SOE_{i}'] = soe
    
    return paac

In [1]:
def calculate_qso(protein_sequence, distance_matrix, weight=0.1, nlag=None):

    amino_acids = "ACDEFGHIKLMNPQRSTVWY"

    if nlag is None:
        nlag = len(protein_sequence) - 1

    QSO_vector = np.zeros(20 + nlag)

    norm_occurrence = np.array([protein_sequence.count(aa) for aa in amino_acids], dtype=float)
    norm_occurrence /= norm_occurrence.sum()

    Sk = np.zeros(nlag)
    for k in range(1, nlag + 1):
        for i in range(len(protein_sequence) - k):
            aa1 = amino_acids.index(protein_sequence[i])
            aa2 = amino_acids.index(protein_sequence[i + k])
            Sk[k - 1] += distance_matrix[aa1, aa2]
        if len(protein_sequence) - k != 0: 
            Sk[k - 1] /= (len(protein_sequence) - k)
        else:
            Sk[k - 1] = 0
    denominator = norm_occurrence.sum() + weight * Sk.sum()
    QSO_vector[:20] = norm_occurrence / denominator

    QSO_vector[20:] = (weight * Sk) / denominator

    return QSO_vector

In [7]:
distance_matrix = np.random.rand(20, 20)
distance_matrix = (distance_matrix + distance_matrix.T) / 2 
np.fill_diagonal(distance_matrix, 0) 
calculate_qso('MAELKSLSGDAY', distance_matrix,nlag=0)

array([0.16666667, 0.        , 0.08333333, 0.08333333, 0.        ,
       0.08333333, 0.        , 0.        , 0.08333333, 0.16666667,
       0.08333333, 0.        , 0.        , 0.        , 0.        ,
       0.16666667, 0.        , 0.        , 0.        , 0.08333333])

In [8]:
for i in range(len(training)):
    protein_sequence = training['Sequence'][i]
    aac = amino_acid_composition(protein_sequence)
    asdc = calculate_asdc(protein_sequence, 1)
    paac = calculate_paac(protein_sequence)
    
    distance_matrix = np.random.rand(20, 20)
    distance_matrix = (distance_matrix + distance_matrix.T) / 2 
    np.fill_diagonal(distance_matrix, 0) 
    qso_feature_vector = calculate_qso(protein_sequence, distance_matrix, nlag=24)
    
    feature_pro = list(asdc.values()) + list(aac.values()) + list(paac.values()) + list(qso_feature_vector)
    if i == 0:
        df = pd.DataFrame([feature_pro])
    else:
        df.loc[i] = feature_pro

In [13]:
df['Label'] = training['Label']
df

,0,1,2,3,4,5,6,7,8,9,...,477,478,479,480,481,482,483,484,485,Label
0,0.004044,0.000578,0.001155,0.006355,0.001155,0.002311,0.000578,0.004044,0.004622,0.003466,...,0.022646,0.021774,0.021876,0.022326,0.022119,0.022840,0.022136,0.021842,0.021633,1
1,0.008448,0.001056,0.002112,0.003168,0.006336,0.007392,0.002112,0.000000,0.003168,0.008448,...,0.022380,0.021570,0.022145,0.022265,0.022516,0.022120,0.022032,0.022376,0.021704,1
2,0.000000,0.003044,0.000000,0.003044,0.000000,0.001522,0.000000,0.000000,0.006088,0.004566,...,0.021909,0.022234,0.023185,0.022064,0.022583,0.023405,0.023008,0.022391,0.022678,1
3,0.021192,0.002649,0.007947,0.011921,0.002649,0.015894,0.000000,0.002649,0.000000,0.015894,...,0.022423,0.020768,0.022295,0.021881,0.022140,0.021656,0.022058,0.021656,0.021421,1
4,0.003494,0.000499,0.002246,0.002246,0.001747,0.003244,0.000998,0.003743,0.001997,0.004742,...,0.022511,0.022664,0.022727,0.023107,0.022857,0.022520,0.022927,0.022486,0.022358,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2513,0.005540,0.000000,0.005540,0.005540,0.000000,0.005540,0.000000,0.011080,0.005540,0.002770,...,0.022552,0.021728,0.022944,0.020155,0.022267,0.022240,0.022330,0.022037,0.022022,0
2514,0.009331,0.000000,0.004666,0.001555,0.001555,0.010886,0.001555,0.001555,0.001555,0.009331,...,0.023177,0.023117,0.022179,0.023702,0.022823,0.023098,0.022438,0.022426,0.022923,0
2515,0.006232,0.001039,0.005972,0.003635,0.004414,0.004674,0.002856,0.002597,0.002337,0.008310,...,0.021991,0.021540,0.021312,0.021837,0.021784,0.021963,0.021811,0.021495,0.021780,0
2516,0.006555,0.000000,0.005827,0.001457,0.000728,0.006555,0.001457,0.002185,0.002185,0.005098,...,0.021514,0.021641,0.021541,0.021668,0.022116,0.021416,0.021662,0.021191,0.021117,0


In [18]:
#df.to_csv("feature_a_testing.csv")

In [15]:
for i in range(len(testing)):
    protein_sequence = testing['Sequence'][i]
    aac = amino_acid_composition(protein_sequence)
    asdc = calculate_asdc(protein_sequence, 1)
    paac = calculate_paac(protein_sequence)
    
    distance_matrix = np.random.rand(20, 20)
    distance_matrix = (distance_matrix + distance_matrix.T) / 2 
    np.fill_diagonal(distance_matrix, 0) 
    qso_feature_vector = calculate_qso(protein_sequence, distance_matrix, nlag=24)
    
    feature_pro = list(asdc.values()) + list(aac.values()) + list(paac.values()) + list(qso_feature_vector)
    if i == 0:
        df = pd.DataFrame([feature_pro])
    else:
        df.loc[i] = feature_pro

In [17]:
df['Label'] = testing['Label']
df

,0,1,2,3,4,5,6,7,8,9,...,477,478,479,480,481,482,483,484,485,Label
0,0.046980,0.002237,0.008949,0.013423,0.008949,0.015660,0.002237,0.002237,0.002237,0.026846,...,0.021301,0.022539,0.023256,0.022649,0.022462,0.023110,0.023425,0.023627,0.022100,1
1,0.004016,0.000000,0.000000,0.004016,0.000000,0.008032,0.008032,0.000000,0.016064,0.000000,...,0.021634,0.023489,0.021734,0.022821,0.020435,0.023375,0.023242,0.020539,0.023697,1
2,0.018963,0.003793,0.002528,0.000000,0.003793,0.016435,0.003793,0.000000,0.003793,0.012642,...,0.021769,0.021863,0.021982,0.021978,0.021011,0.022549,0.022150,0.022434,0.022277,1
3,0.000459,0.000459,0.000000,0.000000,0.006431,0.002297,0.001837,0.004134,0.001837,0.004134,...,0.022183,0.021470,0.020569,0.021377,0.023071,0.021568,0.021535,0.022261,0.022579,1
4,0.001664,0.000555,0.001109,0.004992,0.002773,0.002773,0.001109,0.001664,0.003328,0.007210,...,0.021866,0.021958,0.021303,0.021680,0.021593,0.021996,0.021959,0.022162,0.021850,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,0.004739,0.001354,0.004062,0.004062,0.001354,0.002708,0.001354,0.002031,0.004062,0.007448,...,0.021052,0.020909,0.021390,0.022148,0.021266,0.021746,0.020895,0.020859,0.021457,0
626,0.000000,0.000000,0.005348,0.000000,0.000000,0.010695,0.005348,0.010695,0.005348,0.010695,...,0.021363,0.022239,0.021380,0.023163,0.023497,0.022057,0.022246,0.022066,0.022021,0
627,0.004552,0.001517,0.000000,0.006070,0.004552,0.001517,0.003035,0.003035,0.006070,0.007587,...,0.021813,0.023235,0.022251,0.022151,0.022525,0.022694,0.022418,0.021967,0.023467,0
628,0.005145,0.000643,0.003859,0.003215,0.003215,0.005145,0.000000,0.001929,0.001929,0.007717,...,0.021806,0.022097,0.022798,0.022181,0.022664,0.021996,0.022158,0.022275,0.022089,0


In [6]:
def scale_data(train_data, test_data):
    scaler = StandardScaler()
    scaled_train_data = scaler.fit_transform(train_data)
    scaled_test_data = scaler.transform(test_data)
    return pd.DataFrame(scaled_train_data, columns=train_data.columns), pd.DataFrame(scaled_test_data, columns=test_data.columns)
 

In [4]:
label_data_train = pd.read_csv("feature_a_training.csv", index_col=0)
label_data_test = pd.read_csv("feature_a_testing.csv", index_col=0)

In [7]:
# Separate features and labels
X_train_data = label_data_train.drop('Label', axis=1)
y_train_data = label_data_train['Label']
X_test_data = label_data_test.drop('Label', axis=1)
y_test_data = label_data_test['Label']

In [8]:
scaled_X_train, scaled_X_test = scale_data(X_train_data, X_test_data)

In [12]:
scaled_X_train = pd.concat([scaled_X_train, y_train_data], axis=1)
scaled_X_test = pd.concat([scaled_X_test, y_test_data], axis=1)

In [13]:
scaled_X_test.index = pd.RangeIndex(start=2518, stop=3148)

In [14]:
from pycaret.classification import *
s=setup(data=scaled_X_train, target='Label',test_data=scaled_X_test, fold = 5)

,Description,Value
0,Session id,3415
1,Target,Label
2,Target type,Binary
3,Original data shape,"(3148, 487)"
4,Transformed data shape,"(3148, 487)"
5,Transformed train set shape,"(2518, 487)"
6,Transformed test set shape,"(630, 487)"
7,Numeric features,486
8,Preprocess,True
9,Imputation type,simple


In [15]:
best = compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:02:12
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7451,0.8165,0.7498,0.7453,0.7465,0.4901,0.4915,4.5020
et,Extra Trees Classifier,0.7391,0.8108,0.7244,0.7506,0.7358,0.4782,0.4803,0.1540
rf,Random Forest Classifier,0.7367,0.8065,0.7164,0.7502,0.7313,0.4734,0.4758,0.3080
xgboost,Extreme Gradient Boosting,0.7359,0.8123,0.7347,0.7391,0.7358,0.4718,0.4732,1.6800
ridge,Ridge Classifier,0.7157,0.0000,0.7386,0.7095,0.7221,0.4313,0.4338,0.0440
nb,Naive Bayes,0.7149,0.7697,0.8006,0.6926,0.7396,0.4297,0.4394,0.3780
lr,Logistic Regression,0.7125,0.7816,0.7204,0.7120,0.7146,0.4249,0.4268,0.4820
lda,Linear Discriminant Analysis,0.7117,0.7799,0.7307,0.7066,0.7171,0.4233,0.4253,0.2040
ada,Ada Boost Classifier,0.7069,0.7831,0.7220,0.7027,0.7115,0.4138,0.4148,0.8560
svm,SVM - Linear Kernel,0.6803,0.0000,0.6791,0.6815,0.6797,0.3606,0.3612,0.0900


Processing:   0%|          | 0/69 [00:00<?, ?it/s]


KeyboardInterrupt

